# Examples and Features of Environ Domains

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Cell and EnvironGrid

In [ ]:
from envyron.domains.cell import EnvironGrid

### Minimal Cell 
We start by creating a minimal unitary cell with a 2x2x2 grid. 

In [ ]:
at = np.eye(3)
nr = np.array([2, 2, 2])
minimal_cell = EnvironGrid(at, nr)

The `EnvironGrid` object is a child of the `DirectGrid` class of DFTpy. `DirectGrid` ojbects have a core component `.cell` that is an instance of an ASE (Atomic Simulation Environment) `Cell` object.

In [ ]:
print(type(minimal_cell.cell))
print(minimal_cell.cell)

In [ ]:
minimal_cell.cell.real

On top of the features of the `Cell` object, the `DirectGrid` has all the necessary information on gridpoints, e.g. their position `DirectGrid.r`

In [ ]:
print(type(minimal_cell.r),minimal_cell.r.shape)
print(minimal_cell.r.reshape(3,2*2*2).T)

In [ ]:
minimal_cell.r[:,0,0,0]

In [ ]:
minimal_cell.r[:,0,0,1]

and their distance from the origin `DirectGrid.rr`

In [ ]:
minimal_cell.rr.reshape(2*2*2)

In [ ]:
minimal_cell.corners

In [ ]:
grid = minimal_cell
#
def plot_minimal_cell_xy(origin=np.zeros(3),plot_corners=False):
    fig, ax = plt.subplots()
    ax.set_aspect('equal', 'box')
    ax.set_xlim(-1.1,1.6)
    ax.set_ylim(-1.1,1.6)
    # cell boundaries
    v0 = np.zeros(2)
    v1 = grid.cell.real[0,:2]
    v2 = grid.cell.real[1,:2]
    v3 = grid.cell.real[0,:2] + grid.cell.real[1,:2]
    plt.plot([v0[0],v1[0]],[v0[1],v1[1]],':',color='b')
    plt.plot([v0[0],v2[0]],[v0[1],v2[1]],':',color='b')
    plt.plot([v1[0],v3[0]],[v1[1],v3[1]],':',color='b')
    plt.plot([v2[0],v3[0]],[v2[1],v3[1]],':',color='b')
    # gridpoints
    ax.scatter(grid.r[0,:,:,0],grid.r[1,:,:,0],color='r')
    # some periodic images of the gripoints
    ax.scatter(grid.r[0,:,:,0]+v1[0],grid.r[1,:,:,0]+v1[1],color='r',alpha=0.2)
    ax.scatter(grid.r[0,:,:,0]+v2[0],grid.r[1,:,:,0]+v2[1],color='r',alpha=0.2)
    ax.scatter(grid.r[0,:,:,0]+v3[0],grid.r[1,:,:,0]+v3[1],color='r',alpha=0.2)
    ax.scatter(grid.r[0,:,:,0]-v1[0],grid.r[1,:,:,0]-v1[1],color='r',alpha=0.2)
    ax.scatter(grid.r[0,:,:,0]-v2[0],grid.r[1,:,:,0]-v2[1],color='r',alpha=0.2)
    ax.scatter(grid.r[0,:,:,0]-v3[0],grid.r[1,:,:,0]-v3[1],color='r',alpha=0.2)
    ax.scatter(grid.r[0,:,:,0]+v2[0]-v1[0],grid.r[1,:,:,0]+v2[1]-v1[1],color='r',alpha=0.2)
    ax.scatter(grid.r[0,:,:,0]+v1[0]-v2[0],grid.r[1,:,:,0]+v1[1]-v2[1],color='r',alpha=0.2)
    # corners
    if plot_corners : 
        corners_origin = np.array([[0.,0.],[0.,0.],[0.,0.]])
        plt.quiver(corners_origin[:,0],corners_origin[:,1],grid.corners[grid.corners[:,2]==0][1:,0],grid.corners[grid.corners[:,2]==0][1:,1],color='b',scale=3.2)
    # random point
    if origin.any() :
        plt.scatter(origin[0],origin[1])

plot_minimal_cell_xy()
plt.show()


#### Distance from a Point and Minimum Image Convention

Let us consider a random origin in space and compute the distance of each grid point from such an origin. 

In [ ]:
origin = np.array([0.9, 0.2, 0.0])

In [ ]:
r = grid.r - origin[:, np.newaxis, np.newaxis, np.newaxis]
print(r.reshape(3,8).T)

In [ ]:
plot_minimal_cell_xy(origin)
#
origin_xy = grid.r[:,:,:,0].reshape(3,4).T[:,:2]
r0_xy = -r[:,:,:,0].reshape(3,4).T[:,:2]
plt.quiver(origin_xy[:,0],origin_xy[:,1], r0_xy[:,0], r0_xy[:,1], scale=2.8)
plt.show()

In [ ]:
reciprocal_lattice = grid.get_reciprocal().lattice / 2 / np.pi
print(reciprocal_lattice)

The `np.floor()` integer division will return the vectors to a close image, but staying on the positive side. When visualizing these distance vectors with respect to the individual gridpoints (changing their signs), all the vectors will tend to point towards the bottom right corner. We overshoot at this step to correct later by adding cell vectors (dubbed corners) in the three directions and identifying the point with minimum distance.

In [ ]:
s = np.einsum('lijk,ml->mijk', r, reciprocal_lattice)
s -= np.floor(s)
r = np.einsum('lm,lijk->mijk', grid.lattice, s)

In [ ]:
plot_minimal_cell_xy(origin,plot_corners=True)
#
origin_xy = grid.r[:,:,:,0].reshape(3,4).T[:,:2]
r_xy = -r[:,:,:,0].reshape(3,4).T[:,:2]
plt.quiver(origin_xy[:,0],origin_xy[:,1], r_xy[:,0], r_xy[:,1], scale=2.8)
plt.quiver(origin_xy[:,0],origin_xy[:,1], r0_xy[:,0], r0_xy[:,1], scale=2.8, alpha=0.3)
plt.show()

We iterate over the possible negative images (by adding the corresponding corner vector) to make sure we pick the closest to the gridpoint

In [ ]:
rmin = r
r2min = np.einsum('i...,i...', r, r)
t = r
for corner in grid.corners[1:]: 
    r = t + corner[:,np.newaxis,np.newaxis,np.newaxis]
    r2 = np.einsum('i...,i...', r, r)
    mask = r2 < r2min
    rmin = np.where(mask[np.newaxis, :, :, :], r, rmin)
    r2min = np.where(mask, r2, r2min)


In [ ]:
plot_minimal_cell_xy(origin)
#
origin_xy = grid.r[:,:,:,0].reshape(3,4).T[:,:2]
r_xy = -rmin[:,:,:,0].reshape(3,4).T[:,:2]
plt.quiver(origin_xy[:,0],origin_xy[:,1], r_xy[:,0], r_xy[:,1], scale=2.8)
plt.quiver(origin_xy[:,0],origin_xy[:,1], r0_xy[:,0], r0_xy[:,1], scale=2.8, alpha=0.3)
plt.show()

Note that using the `np.rint()` rounding integer division would provide a quicker approach that would overcome the need for the corners loop. As shown in the following the results are indeed identical to the algorithm that relies on the corners. This approach is used very often in classical Molecular Dynamics, which usually involves orthorombic cells. However, this approach may fail for non-orthorombic cells, which are more common in condensed matter simulations (see later for the hexagonal cell).

In [ ]:
r = grid.r - origin[:, np.newaxis, np.newaxis, np.newaxis]
s = np.einsum('lijk,ml->mijk', r, reciprocal_lattice)
s -= np.rint(s)
r = np.einsum('lm,lijk->mijk', grid.lattice, s)
#
plot_minimal_cell_xy(origin)
#
origin_xy = grid.r[:,:,:,0].reshape(3,4).T[:,:2]
r_xy = -rmin[:,:,:,0].reshape(3,4).T[:,:2]
plt.quiver(origin_xy[:,0],origin_xy[:,1], r_xy[:,0], r_xy[:,1], scale=2.8)
plt.show()

#### Distances in Lower Dimensions

##### Distance from a Line

In [ ]:
dim = 1 # line passing throught the origin
axis = 1 # oriented along the i-th axis, i = 0, 1, 2
origin = np.array([0.9, 0.2, 0.0])

In [ ]:
r = grid.r - origin[:, np.newaxis, np.newaxis, np.newaxis]
s = np.einsum('lijk,ml->mijk', r, reciprocal_lattice)
s -= np.floor(s)
r = np.einsum('lm,lijk->mijk', grid.lattice, s)

# determines the direction of the line
n = grid.cell[axis,:]
# removes the component directed along n
r = r - np.einsum('jkl,i->ijkl',np.einsum('ijkl,i->jkl',r,n),n)

# pre-corner-check results
rmin = r
r2min = np.einsum('i...,i...', r, r)

# check against corner shifts
t = r
for corner in grid.corners[1:]:
    r = t + corner[:,np.newaxis,np.newaxis,np.newaxis]
    r = r - np.einsum('jkl,i->ijkl',np.einsum('ijkl,i->jkl',r,n),n)
    r2 = np.einsum('i...,i...', r, r)
    mask = r2 < r2min
    rmin = np.where(mask[np.newaxis, :, :, :], r, rmin)
    r2min = np.where(mask, r2, r2min)

In [ ]:
plot_minimal_cell_xy(origin)
#
origin_xy = grid.r[:,:,:,0].reshape(3,4).T[:,:2]
r_xy = -rmin[:,:,:,0].reshape(3,4).T[:,:2]
plt.quiver(origin_xy[:,0],origin_xy[:,1], r_xy[:,0], r_xy[:,1], scale=2.8)
plt.show()

##### Distance from a Plane

In [ ]:
dim = 2 # line passing throught the origin
axis = 1 # oriented perpendicular to the i-th axis, i = 0, 1, 2
origin = np.array([0.9, 0.2, 0.0])

In [ ]:
r = grid.r - origin[:, np.newaxis, np.newaxis, np.newaxis]
s = np.einsum('lijk,ml->mijk', r, reciprocal_lattice)
s -= np.floor(s)
r = np.einsum('lm,lijk->mijk', grid.lattice, s)

# determine the two directions of the plane
n1, n2 = grid.cell[np.arange(3)!=axis,:]
# take the cross product to get the perpendicular direction
n = np.cross(n2,n1)
# only keep the component directed along n
r = np.einsum('jkl,i->ijkl',np.einsum('ijkl,i->jkl',r,n),n)

# pre-corner-check results
rmin = r
r2min = np.einsum('i...,i...', r, r)

# check against corner shifts
t = r
for corner in grid.corners[1:]:
    r = t + corner[:,np.newaxis,np.newaxis,np.newaxis]
    r = np.einsum('jkl,i->ijkl',np.einsum('ijkl,i->jkl',r,n),n)
    r2 = np.einsum('i...,i...', r, r)
    mask = r2 < r2min
    rmin = np.where(mask[np.newaxis, :, :, :], r, rmin)
    r2min = np.where(mask, r2, r2min)

In [ ]:
plot_minimal_cell_xy(origin)
#
origin_xy = grid.r[:,:,:,0].reshape(3,4).T[:,:2]
r_xy = -rmin[:,:,:,0].reshape(3,4).T[:,:2]
plt.quiver(origin_xy[:,0],origin_xy[:,1], r_xy[:,0], r_xy[:,1], scale=2.8)
plt.show()

#### Distances and Translational Symmetry

It is important that the computed distances do not depend on arbitrary translations of the origin by a lattice vector. The use of scaled coordinates and `np.floor()` is to ensure that the image closest to the gridpoints is selected. We can add an arbitrary shift by a lattice vectore to our point

In [ ]:
shift = [1, -1, 0] # shift the point by a lattice vector
origin_shifted = origin + np.dot(grid.cell.T,np.array(shift))
print(origin_shifted)

and the results are not affected

In [ ]:
plot_minimal_cell_xy(origin_shifted)
plt.xlim(-1.1,2.1)
#
rmin, r2min = grid.get_min_distance(origin_shifted)
origin_xy = grid.r[:,:,:,0].reshape(3,4).T[:,:2]
r_xy = -rmin[:,:,:,0].reshape(3,4).T[:,:2]
plt.quiver(origin_xy[:,0],origin_xy[:,1], r_xy[:,0], r_xy[:,1], scale=3.2)
plt.show()

In [ ]:
plot_minimal_cell_xy(origin_shifted)
plt.xlim(-1.1,2.1)
#
rmin, r2min = grid.get_min_distance(origin_shifted,dim=1,axis=0)
origin_xy = grid.r[:,:,:,0].reshape(3,4).T[:,:2]
r_xy = -rmin[:,:,:,0].reshape(3,4).T[:,:2]
plt.quiver(origin_xy[:,0],origin_xy[:,1], r_xy[:,0], r_xy[:,1], scale=3.2)
plt.show()

### Hexagonal Cell

In [ ]:
at = np.eye(3) * 1
at[1, 0] = 0.5
at[1, 1] *= np.sqrt(3) * 0.5
nr = np.array([2, 2, 2])
hexagonal_cell = EnvironGrid(at, nr, label='system')

In [ ]:
grid = hexagonal_cell
#
fig, ax = plt.subplots()
ax.set_aspect('equal', 'box')
ax.set_xlim(-1.7,2.6)
ax.set_ylim(-1.1,1.6)
# cell boundaries
v0 = np.zeros(2)
v1 = grid.cell.real[0,:2]
v2 = grid.cell.real[1,:2]
v3 = grid.cell.real[0,:2] + grid.cell.real[1,:2]
plt.plot([v0[0],v1[0]],[v0[1],v1[1]],':',color='b')
plt.plot([v0[0],v2[0]],[v0[1],v2[1]],':',color='b')
plt.plot([v1[0],v3[0]],[v1[1],v3[1]],':',color='b')
plt.plot([v2[0],v3[0]],[v2[1],v3[1]],':',color='b')
# gridpoints
ax.scatter(grid.r[0,:,:,0],grid.r[1,:,:,0],color='r')
# some periodic images of the gripoints
ax.scatter(grid.r[0,:,:,0]+v1[0],grid.r[1,:,:,0]+v1[1],color='r',alpha=0.2)
ax.scatter(grid.r[0,:,:,0]+v2[0],grid.r[1,:,:,0]+v2[1],color='r',alpha=0.2)
ax.scatter(grid.r[0,:,:,0]+v3[0],grid.r[1,:,:,0]+v3[1],color='r',alpha=0.2)
ax.scatter(grid.r[0,:,:,0]-v1[0],grid.r[1,:,:,0]-v1[1],color='r',alpha=0.2)
ax.scatter(grid.r[0,:,:,0]-v2[0],grid.r[1,:,:,0]-v2[1],color='r',alpha=0.2)
ax.scatter(grid.r[0,:,:,0]-v3[0],grid.r[1,:,:,0]-v3[1],color='r',alpha=0.2)
ax.scatter(grid.r[0,:,:,0]+v2[0]-v1[0],grid.r[1,:,:,0]+v2[1]-v1[1],color='r',alpha=0.2)
ax.scatter(grid.r[0,:,:,0]+v1[0]-v2[0],grid.r[1,:,:,0]+v1[1]-v2[1],color='r',alpha=0.2)
# corners
plt.scatter(grid.corners[grid.corners[:,2]==0][1:,0],grid.corners[grid.corners[:,2]==0][1:,1])
# random point
plt.scatter([1.1],[0.1])
plt.show()

In [ ]:
origin = np.array([1.1,0.1,0.])

In [ ]:
dr,dr2 = grid.get_min_distance(origin,dim=1,axis=1)

In [ ]:
dr.reshape(3,8).T

In [ ]:
dr2.reshape(8)